In [ ]:
path_consultoria = '/content/drive/MyDrive/Consultoria/Final'

In [ ]:
!cp -r {path_consultoria}/UR5Inv /content/

In [ ]:
!pip install optuna

In [ ]:
from UR5Inv.FNN import FNNInversaUR5

In [ ]:
def comparar_modelos(nombre_modelo, ruta_datos, modelo, ruta_csv=None):
    """
    Crea o actualiza un archivo CSV con métricas de comparación entre diferentes modelos.

    Parámetros:
        nombre_modelo (str): Nombre identificador del modelo (ej: "Aleatorio_FNN").
        ruta_datos (str): Ruta al archivo de datos utilizado para entrenar el modelo.
        modelo (FNNInversaUR5): Instancia del modelo entrenado para evaluar.
        ruta_csv (str, opcional): Ruta donde guardar el archivo CSV. Por defecto se guarda
                                 en el mismo directorio que los datos originales.
    """
    import pandas as pd
    import numpy as np
    import os
    import time
    import torch

    # Determinar ruta del CSV si no se especifica
    if ruta_csv is None:
        directorio_datos = os.path.dirname(ruta_datos)
        ruta_csv = os.path.join(directorio_datos, "comparacion_modelos.csv")

    # Obtener resultados de evaluación
    resultados = modelo.evaluar_total()

    # Calcular errores L1 por articulación
    modelo.modelo_final.eval()
    with torch.no_grad():
        pred_total = modelo.modelo_final(torch.tensor(modelo.X, dtype=torch.float32).to(modelo.device)).cpu().numpy()

    errores_L1 = np.mean(np.abs(modelo.Y - pred_total), axis=0)

    # Calcular tiempo de inferencia promedio (1000 muestras)
    tiempos = []
    n_muestras = min(1000, len(modelo.X))
    indices = np.random.choice(len(modelo.X), n_muestras, replace=False)

    for idx in indices:
        x = torch.tensor(modelo.X[idx:idx+1], dtype=torch.float32).to(modelo.device)

        inicio = time.time()
        with torch.no_grad():
            modelo.modelo_final(x)
        fin = time.time()

        tiempos.append(fin - inicio)

    tiempo_inf = np.mean(tiempos) * 1000  # ms

    # Calcular tamaño del modelo (parámetros)
    parametros = sum(p.numel() for p in modelo.modelo_final.parameters())

    # Crear diccionario con las métricas - USANDO RADIANES en lugar de grados
    metricas = {
        'modelo': nombre_modelo,
        'MSE_train': resultados['train']['mse_rad'] if resultados['train'] else None,
        'MSE_test': resultados['test']['mse_rad'] if resultados['test'] else None,
        'MSE_global': resultados['total']['mse_rad'],
        'tamaño_modelo': parametros,
        'tiempo_inf': tiempo_inf
    }

    # Añadir errores L1 por articulación
    for i, error in enumerate(errores_L1):
        metricas[f'Errorq{i}L1'] = error

    # Crear o actualizar el archivo CSV
    try:
        # Intentar leer el archivo existente
        df_existente = pd.read_csv(ruta_csv)
        # Verificar si este modelo ya existe
        if nombre_modelo in df_existente['modelo'].values:
            # Si existe, actualizarlo
            df_existente = df_existente[df_existente['modelo'] != nombre_modelo]

        # Añadir el nuevo modelo
        df_nueva_fila = pd.DataFrame([metricas])
        df_final = pd.concat([df_existente, df_nueva_fila], ignore_index=True)

    except (FileNotFoundError, pd.errors.EmptyDataError):
        # Si el archivo no existe o está vacío, crear uno nuevo
        df_final = pd.DataFrame([metricas])

    # Guardar el archivo
    df_final.to_csv(ruta_csv, index=False)
    print(f"Comparación actualizada en: {ruta_csv}")

    return df_final

## Datos comandos

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/2dos/datos_comandos.csv"

modelo = FNNInversaUR5(
    ruta_datos= datos,
    test_size=0.3,
    random_state=42,
)

print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=12)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.evaluar_total()
modelo.guardar("Comandos")

comparar_modelos(
    nombre_modelo="Comandos_FNN",
    ruta_datos= datos,
    modelo=modelo
)

!cp -r /content/Comandos /content/drive/MyDrive/Consultoria/Final


## Datos Aleatorios

In [ ]:
datos = r"/content/drive/MyDrive/Consultoria/Final/Data/3ros/Aleatorio.csv"

modelo = FNNInversaUR5(
    ruta_datos=datos,
    test_size=0.1,
    random_state=42,
)

print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=50)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.evaluar_total()
modelo.guardar("Aleatorios")

comparar_modelos(
    nombre_modelo="Aleatorio_FNN",
    ruta_datos="/content/drive/MyDrive/Consultoria/Final/Data/3ros/Aleatorio.csv",
    modelo=modelo
)

!cp -r /content/Aleatorios /content/drive/MyDrive/Consultoria/Final



## Trayectorias

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/Gen1/openaxes-example-robot-dataset/robot/Complete.csv"

modelo = FNNInversaUR5(
    ruta_datos= datos,
    test_size=0.3,
    random_state=42,
)

print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=12)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.evaluar_total()
modelo.guardar("Trayectorias")

comparar_modelos(
    nombre_modelo="Trayectorias_FNN",
    ruta_datos= datos,
    modelo=modelo
)

!cp -r /content/Trayectorias /content/drive/MyDrive/Consultoria/Final

## PesosVel

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/Gen2/NISTdata.csv"

modelo = FNNInversaUR5(
    ruta_datos= datos,
    test_size=0.3,
    random_state=42,
)

print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=8)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.evaluar_total()
modelo.guardar("PesosVel")

comparar_modelos(
    nombre_modelo="PesosVel_FNN",
    ruta_datos= datos,
    modelo=modelo
)

!cp -r /content/PesosVel /content/drive/MyDrive/Consultoria/Final